# Create AWS Lambda Function as Kinesis Analytics Destination


In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

iam = boto3.Session().client(service_name='iam', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
account_id = sts.get_caller_identity()['Account']

lam = boto3.Session().client(service_name='lambda', region_name=region)

# Specify AWS Lambda Function Name

In [ ]:
lambda_fn_name = 'DeliverKinesisAnalyticsToCloudWatch'

# Create AWS Lambda IAM Role

In [ ]:
iam_lambda_role_name = 'DSOAWS_Lambda'

In [ ]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "kinesisanalytics.amazonaws.com"
        },
            "Action": "sts:AssumeRole"
        }
    ]
}

In [ ]:
import time

from botocore.exceptions import ClientError

try:
    iam_role_lambda = iam.create_role(
        RoleName=iam_lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Lambda Role'
    )
    print('Role succesfully created.')
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_lambda = iam.get_role(RoleName=iam_lambda_role_name)
        print('Role already exists. That is OK.')
    else:
        print('Unexpected error: %s' % e)
        
time.sleep(10)

In [ ]:
iam_role_lambda_name = iam_role_lambda['Role']['RoleName']
print('Role Name: {}'.format(iam_role_lambda_name))

In [ ]:
iam_role_lambda_arn = iam_role_lambda['Role']['Arn']
print('Role ARN: {}'.format(iam_role_lambda_arn))

# Create AWS Lambda IAM Policy

In [ ]:
lambda_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "UseLambdaFunction",
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Resource": "arn:aws:lambda:{}:{}:function:*".format(region, account_id)
        },
        {
            "Effect": "Allow",
            "Action": "cloudwatch:*",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:{}:{}:*".format(region, account_id)
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:{}:{}:log-group:/aws/lambda/*".format(region, account_id)
        }
    ]
}

In [ ]:
print(json.dumps(lambda_policy_doc, indent=4, sort_keys=True, default=str))

In [ ]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_lambda_name,
    PolicyName='DSOAWS_LambdaPolicy',
    PolicyDocument=json.dumps(lambda_policy_doc)
)

time.sleep(10)

In [ ]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

# Create AWS Lambda Function

In [ ]:
# Loads the zip file as binary code. 
with open('src/DeliverKinesisAnalyticsToCloudWatch.zip', 'rb') as f: 
    code = f.read()

In [ ]:
response = lam.create_function(
    FunctionName='{}'.format(lambda_fn_name),
    Runtime='python2.7',
    Role='{}'.format(iam_role_lambda_arn),
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': code
    },
    Description='Deliver output records from Kinesis Analytics application to CloudWatch.',
    Timeout=60,
    MemorySize=128,
    Publish=True,
#    VpcConfig={
#        'SubnetIds': [
#            'string',
#        ],
#        'SecurityGroupIds': [
#            'string',
#        ]
#    },
#    DeadLetterConfig={
#        'TargetArn': 'string'
#    },
#    Environment={
#        'Variables': {
#            'string': 'string'
#        }
#    },
#    KMSKeyArn='string',
#    TracingConfig={
#        'Mode': 'Active'|'PassThrough'
#    },
#    Tags={
#        'string': 'string'
#    },
#    Layers=[
#        'string',
#    ],
#    FileSystemConfigs=[
#        {
#            'Arn': 'string',
#            'LocalMountPath': 'string'
#        },
#    ]
)

In [ ]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

In [ ]:
lambda_fn_arn = response['FunctionArn']
print(lambda_fn_arn)

# Store Variables for Next Notebooks

In [ ]:
%store iam_lambda_role_name

In [ ]:
%store iam_role_lambda_arn

In [ ]:
%store lambda_fn_name

In [ ]:
%store lambda_fn_arn

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();